In [ ]:
# default_exp layers
# all_slow

# Layers

> Pytorch model layers

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from mrl.imports import *
from mrl.torch_imports import *

In [ ]:
# export

class Linear(nn.Module):
    def __init__(self, d_in, d_out, act=True, bn=False, dropout=0.):
        super().__init__()
        
        layers = [nn.Linear(d_in, d_out)]
        
        if bn:
            layers.append(nn.BatchNorm1d(d_out))
            
        if act:
            layers.append(nn.ReLU())
            
        if dropout>0.:
            layers.append(nn.Dropout(p=dropout))
            
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x)

In [ ]:
# export

class LSTM(nn.Module):
    def __init__(self, d_embedding, d_hidden, d_output, n_layers, 
                 bidir=False, dropout=0., batch_first=True):
        super().__init__()
        
        self.d_embedding = d_embedding
        self.d_hidden = d_hidden
        self.d_output = d_output
        self.n_layers = n_layers
        self.bidir = bidir
        self.n_dir = 1 if not bidir else 2
        self.batch_first = batch_first
        
        self.lstms = []
        self.hidden_sizes = []
        
        for l in range(n_layers):
            input_size = d_embedding if l==0 else d_hidden
            output_size = d_output if l==n_layers-1 else d_hidden
            
            hidden_size = (self.n_dir, 1, output_size)
            self.hidden_sizes.append(hidden_size)
            
            lstm = nn.LSTM(input_size, output_size, 1, batch_first=batch_first, 
                           dropout=dropout, bidirectional=bidir)
            self.lstms.append(lstm)
            
        self.lstms = nn.ModuleList(self.lstms)
        
    def forward(self, x, hiddens=None):
        
        bs = x.shape[0] if self.batch_first else x.shape[1]
        
        if hiddens is None:
            hiddens = self.get_new_hidden(bs)
            hiddens = [(i[0].to(x.device), i[1].to(x.device)) for i in hiddens]
            
        new_hiddens = []
        for i, lstm in enumerate(self.lstms):
            x, (h,c) = lstm(x, hiddens[i])
            new_hiddens.append((h.detach(), c.detach()))
            
        return x, new_hiddens
            
    def get_new_hidden(self, bs):
        hiddens = []
        for hs in self.hidden_sizes:
            h = torch.zeros(hs).repeat(1,bs,1)
            c = torch.zeros(hs).repeat(1,bs,1)
            hiddens.append((h,c))
        
        return hiddens

In [ ]:
# export

class LSTMLM(nn.Module):
    def __init__(self, d_vocab, d_embedding, d_hidden, n_layers, pad_idx, 
                 lstm_drop=0., bos_idx=0, bidir=False):
        super().__init__()
        
        self.embedding = nn.Embedding(d_vocab, d_embedding)
        self.lstm = LSTM(d_embedding, d_hidden, d_embedding, n_layers, bidir=bidir, dropout=lstm_drop)
        self.head = Linear(d_embedding, d_vocab, act=False, bn=False, dropout=0.)
        self.bos_idx = bos_idx
        
    def forward(self, x):
        x = self.embedding(x)
        x, hiddens = self.lstm(x)
        self.last_hidden = hiddens
        x = self.head(x)
        return x
    
    def sample(self, bs, sl, multinomial=True):
        
        preds = idxs = torch.tensor([vocab.stoi['bos']]*bs).long().unsqueeze(-1) # todo - cuda
        lps = []

        hiddens = self.lstm.get_new_hidden(bs)
        
        for i in range(sl):
            x = self.embedding(idxs)
            x, hiddens = self.lstm(x, hiddens)
            x = self.head(x)

            log_probs = F.log_softmax(x, -1).squeeze(1)
            probs = log_probs.detach().exp()
            
            if multinomial:
                idxs = torch.multinomial(probs, 1)
            else:
                idxs = x.argmax(-1)
                
            lps.append(torch.gather(log_probs, 1, idxs))
            
            preds = torch.cat([preds, idxs], -1)
            
        return preds[:, 1:], torch.cat(lps,-1)
    
    def sample_no_grad(self, bs, sl, multinomial=True):
        with torch.no_grad():
            return self.sample(bs, sl, multinomial=multinomial)

In [ ]:
from mrl.dataloaders import *
from mrl.chem import *
from mrl.core import *

//anaconda3/envs/mrl/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.
  return f(*args, **kwds)


In [ ]:
vocab = CharacterVocab(SMILES_CHAR_VOCAB)

In [ ]:
model = LSTMLM(len(vocab.itos), 256, 1024, 3, vocab.stoi['pad'])

In [ ]:
model.load_state_dict(torch.load('untracked_files/smiles_lm.pt'))

<All keys matched successfully>

In [ ]:
from mrl.templates import *

In [ ]:
template = Template([ValidityFilter(), SingleCompoundFilter()],
                    [QEDFilter(None, None, score=PassThroughScore())], fail_score=-1.)

In [ ]:
preds, lps = model.sample_no_grad(100,100)
smiles = [vocab.reconstruct(i) for i in preds]

In [ ]:
smiles[0]

'O=Cc1ccc(OC(F)(F)C(=O)Nc2ncc(-c3cccnc3)s2)cc1'

In [ ]:
os.environ['ncpus'] = '0'

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
all_scores = []

for i in range(10):
    print(i)
    preds, lps = model.sample(64,90)
    smiles = [vocab.reconstruct(i) for i in preds]
    scores = np.array(template.eval_mols(smiles))
    all_scores.append(scores)
    scores = scores - scores.mean()
    
    opt.zero_grad()
    loss = -(lps.sum(-1))*torch.from_numpy(scores)
    loss = loss.mean()
    loss.backward()
    opt.step()

0
1
2
3
4
5
6
7
8
9


In [ ]:
[i.mean() for i in all_scores]

[0.7030050660666866,
 0.7008088306317556,
 0.7322921057256926,
 0.7426690798871887,
 0.6456491829284243,
 0.7341448932857702,
 0.7397209582560801,
 0.7478360480069184,
 0.7648231396284492,
 0.7151239887071124]

In [ ]:
# hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_chem.ipynb.
Converted 02_template.filters.ipynb.
Converted 03_template.template.ipynb.
Converted 04_template.blocks.ipynb.
Converted 05_torch_core.ipynb.
Converted 06_layers.ipynb.
Converted 07_dataloaders.ipynb.
Converted index.ipynb.
Converted template.overview.ipynb.
Converted tutorials.ipynb.
Converted tutorials.structure_enumeration.ipynb.
Converted tutorials.template.advanced.ipynb.
Converted tutorials.template.beginner.ipynb.
Converted tutorials.template.intermediate.ipynb.
